<a href="https://colab.research.google.com/github/SaqlainHussainShah/Cross-site-scripting-attacks/blob/master/XSS_V2_CNN_ASCII_128_conv_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import glob
import time
import pandas as pd
# from xml.dom import minidom

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords 

import tensorflow as tf
from tensorflow import keras
import os
import matplotlib.pyplot as plt



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
import cv2

In [0]:
df = pd.read_csv('./XSS.csv', encoding='utf-8-sig')

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16317 entries, 0 to 16316
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  16317 non-null  int64 
 1   Sentence    16317 non-null  object
 2   Label       16317 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 382.6+ KB


In [18]:
df.head()

,Unnamed: 0,Sentence,Label
0,0,['canfranc' 8 'norm' 'norm'],0
1,1,"<abbr oncut=""alert(1)"" contenteditable>test</a...",1
2,2,"<picture onmouseover=""alert(1)"">test</picture>",1
3,3,"<div role=""navigation"" class=""navbox"" aria-lab...",0
4,4,"<li id=""cite_note-FOOTNOTEDomingos2015210-207""...",0


In [19]:
df.head()

,Unnamed: 0,Sentence,Label
0,0,['canfranc' 8 'norm' 'norm'],0
1,1,"<abbr oncut=""alert(1)"" contenteditable>test</a...",1
2,2,"<picture onmouseover=""alert(1)"">test</picture>",1
3,3,"<div role=""navigation"" class=""navbox"" aria-lab...",0
4,4,"<li id=""cite_note-FOOTNOTEDomingos2015210-207""...",0


In [0]:
df=df[df.columns[-2:]]  # Only get sentence and labels

In [21]:
df.head()

,Sentence,Label
0,['canfranc' 8 'norm' 'norm'],0
1,"<abbr oncut=""alert(1)"" contenteditable>test</a...",1
2,"<picture onmouseover=""alert(1)"">test</picture>",1
3,"<div role=""navigation"" class=""navbox"" aria-lab...",0
4,"<li id=""cite_note-FOOTNOTEDomingos2015210-207""...",0


In [22]:
sentences=df['Sentence'].values
sentences[1]

'<abbr oncut="alert(1)" contenteditable>test</abbr>'

In [23]:
print(len(sentences))

16317


In [0]:
df_xss = df[df['Label'] == 1]

In [25]:
df_xss.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8004 entries, 1 to 16316
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  8004 non-null   object
 1   Label     8004 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 187.6+ KB


In [0]:
df_benign = df[df['Label'] == 0]


In [27]:
df_benign.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8313 entries, 0 to 16308
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  8313 non-null   object
 1   Label     8313 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 194.8+ KB


In [0]:
# Convert to ASCII

def convert_to_ascii(sentence):
    sentence_ascii=[]

    for i in sentence:
        
        
        """Some characters have values very big e.d 8221 adn some are chinese letters
        I am removing letters having values greater than 8222 and for rest greater 
        than 128 and smaller than 8222 assigning them values so they can easily be normalized"""
       
        if(ord(i)<8222):      # ” has ASCII of 8221
            
            if(ord(i)==8217): # ’  :  8217
                sentence_ascii.append(134)
            
            
            if(ord(i)==8221): # ”  :  8221
                sentence_ascii.append(129)
                
            if(ord(i)==8220): # “  :  8220
                sentence_ascii.append(130)
                
                
            if(ord(i)==8216): # ‘  :  8216
                sentence_ascii.append(131)
                
            if(ord(i)==8217): # ’  :  8217
                sentence_ascii.append(132)
            
            if(ord(i)==8211): # –  :  8211
                sentence_ascii.append(133)
                
                
            """
            If values less than 128 store them else discard them
            """
            if (ord(i)<=128):
                    sentence_ascii.append(ord(i))
    
            else:
                    pass
            

    zer=np.zeros((10000))

    for i in range(len(sentence_ascii)):
        zer[i]=sentence_ascii[i]

    zer.shape=(100, 100)


#     plt.plot(image)
#     plt.show()
    return zer

In [0]:
arr=np.zeros((len(sentences),100,100))

for i in range(len(sentences)):
    
    image=convert_to_ascii(sentences[i])

    x=np.asarray(image,dtype='float')
    image =  cv2.resize(x, dsize=(100,100), interpolation=cv2.INTER_CUBIC)
    image/=128

    
#     if i==1:
#         plt.plot(image)
#         plt.show()    
    arr[i]=image

In [0]:
data = arr.reshape(arr.shape[0], 100, 100, 1)

In [31]:
data.shape

(16317, 100, 100, 1)

In [0]:
y=df['Label'].values

In [0]:


# Split into train/test data
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(data,y, test_size=0.2, random_state=42)



In [34]:
trainX.shape


(13053, 100, 100, 1)

In [36]:


# import libraries for Making Model
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D,Flatten,Dropout,MaxPool2D, BatchNormalization



Using TensorFlow backend.


In [0]:
model=tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(64,(3,3), activation=tf.nn.relu, input_shape=(100,100,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(256,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    
])

In [38]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 256)       295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 256)       0         
_________________________________________________________________
flatten (Flatten)            (None, 25600)             0

In [0]:

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.97):
            print(" \n Reached 99% + validation accuracy")
            self.model.stop_training=True
callbacks = myCallback()

In [40]:


batch_size = 128
num_epoch = 10
#model training
model_log = model.fit(trainX, trainY,
          batch_size=batch_size,
          epochs=num_epoch,
          verbose=1,
          validation_data=( testX,  testY)
#                        callbacks=[callbacks]
                     )



Epoch 1/10
102/102 [==============================] - 10s 95ms/step - loss: 0.4702 - accuracy: 0.7708 - val_loss: 0.3563 - val_accuracy: 0.8333
Epoch 2/10
102/102 [==============================] - 8s 82ms/step - loss: 0.2089 - accuracy: 0.9202 - val_loss: 0.1122 - val_accuracy: 0.9672
Epoch 3/10
102/102 [==============================] - 8s 83ms/step - loss: 0.0986 - accuracy: 0.9674 - val_loss: 0.0882 - val_accuracy: 0.9715
Epoch 4/10
102/102 [==============================] - 8s 82ms/step - loss: 0.0775 - accuracy: 0.9744 - val_loss: 0.0737 - val_accuracy: 0.9773
Epoch 5/10
102/102 [==============================] - 8s 82ms/step - loss: 0.0602 - accuracy: 0.9815 - val_loss: 0.0581 - val_accuracy: 0.9838
Epoch 6/10
102/102 [==============================] - 8s 83ms/step - loss: 0.0519 - accuracy: 0.9835 - val_loss: 0.0540 - val_accuracy: 0.9844
Epoch 7/10
102/102 [==============================] - 8s 83ms/step - loss: 0.0501 - accuracy: 0.9823 - val_loss: 0.0752 - val_accuracy: 0.977

In [0]:
pred=model.predict(testX)

In [0]:

for i in range(len(pred)):
    if pred[i]>0.5:
        pred[i]=1
    elif pred[i]<=0.5:
        pred[i]=0

In [43]:

true=0
false=0

for i in range(len(pred)):
    if pred[i] == testY[i]:
        true+=1
    else:
        false+=1
        
print("correct predicted :: ", true)
print("false prediction :: ", false)

correct predicted ::  3215
false prediction ::  49


In [44]:
attack=0
benign=0
for i in range(len(testY)):
    if testY[i]==1:
        attack+=1
    else:
        benign+=1

print("Attack data in test set :: ", attack)
print(" Benign data in test set :: ", benign)

Attack data in test set ::  1615
 Benign data in test set ::  1649


In [0]:

def accuracy_function(tp,tn,fp,fn):
    
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    
    return accuracy

In [0]:
def precision_function(tp,fp):
    
    precision = tp / (tp+fp)
    
    return precision

In [0]:
def recall_function(tp,fn):
    
    recall=tp / (tp+fn)
    
    return recall

In [0]:
def confusion_matrix(truth,predicted):
    
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    
    for true,pred in zip(truth,predicted):
        if true == 1:
            if pred == true:
                true_positive += 1
            elif pred != true:
                false_negative += 1

        elif true == 0:
            if pred == true:
                true_negative += 1
            elif pred != true:
                false_positive += 1
            
    accuracy=accuracy_function(true_positive, true_negative, false_positive, false_negative)
    precision=precision_function(true_positive, false_positive)
    recall=recall_function(true_positive, false_negative)
    
    return (accuracy,
            precision,
           recall)

In [0]:
accuracy,precision,recall=confusion_matrix(testY,pred)

In [50]:


print(" For CNN  \n Accuracy : {0} \n Precision : {1} \n Recall : {2} \n".format(accuracy, precision, recall))



 For CNN  
 Accuracy : 0.9849877450980392 
 Precision : 0.9833333333333333 
 Recall : 0.9863777089783282 

